# Build the Neural Network

Tutorial link: https://docs.pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(dim=1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='mps:0')


In [10]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [12]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0329,  0.3010, -0.2635, -0.5516,  0.3659, -0.3478,  0.5506,  0.1469,
         -0.3484, -0.2039, -0.6153,  0.2456, -0.4014, -0.4263, -0.1628, -0.4493,
         -0.1143,  0.0615, -0.3130,  0.2259],
        [ 0.1474,  0.3180,  0.2979, -0.1262,  0.3660,  0.0465,  0.5044,  0.6412,
         -0.1889, -0.0018, -0.2369,  0.4602, -0.2591, -0.5324, -0.1704, -0.3426,
          0.0164, -0.2318, -0.0639,  0.4911],
        [-0.3221,  0.3654, -0.1084, -0.4888,  0.2033,  0.1071,  0.5139,  0.4614,
          0.2176,  0.1485, -0.3317,  0.5206, -0.2113, -0.3750,  0.1435, -0.3648,
          0.0505,  0.1729, -0.1902,  0.1730]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.3010, 0.0000, 0.0000, 0.3659, 0.0000, 0.5506, 0.1469, 0.0000,
         0.0000, 0.0000, 0.2456, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0615,
         0.0000, 0.2259],
        [0.1474, 0.3180, 0.2979, 0.0000, 0.3660, 0.0465, 0.5044, 0.6412, 0.0000,
         0.0000, 0.0000, 0.4602, 0.0000, 0.0000, 0.00

In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [15]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [16]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0040, -0.0209, -0.0164,  ...,  0.0208, -0.0153,  0.0332],
        [ 0.0239, -0.0093, -0.0133,  ..., -0.0083, -0.0132, -0.0279]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0094, -0.0291], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0426,  0.0157, -0.0128,  ..., -0.0376,  0.0377,  0.0173],
        [ 0.0385, -0.0236, -0.0285,  ..., -0.0316,  0.0085,  0.0026]],
       device='mps:0', grad_fn=<Slice